In [ ]:
#conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1

In [ ]:
#pip install tensorflow==2.11

In [ ]:
#conda install -c conda-forge cudatoolkit=11.2

In [5]:
pip install tensorflow_addons

/bin/bash: /home/ma00048/miniconda3/envs/DL/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 15.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]
Pandas 1.5.3
Scikit-Learn 1.2.2
GPU is available


In [7]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, KFold
import sklearn.preprocessing

import time
import sklearn
import numpy as np
import os
import sys
from joblib import parallel_backend

In [8]:
# define a list of datasets
datasets = ["PHM2022_Multivar", "PHM2022_Univar_PDIN"]
datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    print(Dataset.shape)
    

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")

(53785, 749, 3)
(53785, 749, 1)


In [9]:
# change this directory for your machine
root_dir = './'

# define the number of folds
n_folds = 5

# perform cross-validation for each dataset and algorithm combination
for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    


    # Create a folder for results
    results_path = root_dir + "Results/" + Dataset_name
    if os.path.exists(results_path):
        pass
    else:
        try:
            os.makedirs(results_path)
        except:
            # in case another machine created the path meanwhile !:(
            pass

        
    t_total = time.time() ##Start timing
    


    print(f"\n The dataset shape is:{Dataset.shape}")
    print(f"\n The number of data samples (N) is:{Dataset.shape[0]}")
    print(f"\n The number of TS length (T) is:{Dataset.shape[1]}")
    print(f"\n The number of TS dimention (M) is:{Dataset.shape[2]}")

    nb_classes = len(np.unique(Labels, axis=0))
    
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    Labels = enc.fit_transform(Labels.reshape(-1, 1)).toarray()


    kf = KFold(n_splits=n_folds, shuffle=True)
    accuracy_scores = []
    f1_scores = []
    confusion_matrices = []
    report_list = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(Dataset)):
        # split the data into training and testing sets
        X_train, X_test = Dataset[train_idx], Dataset[test_idx]
        y_train, y_test = Labels[train_idx], Labels[test_idx]
        
        # save orignal y because later we will use binary
        y_true = np.argmax(y_test, axis=1)
        
        input_shape = X_train.shape[1:]
        
        ## Create Classification module
        from dl4tsc.classifiers import encoder
        classifier = encoder.Classifier_ENCODER(output_directory=results_path, input_shape=input_shape,
                                        nb_classes=nb_classes, verbose=True)
        
        # fit the algorithm on the training data
        classifier.fit(X_train, y_train, X_test, y_test, y_true)
            
        # calculate the evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        print(accuracy)

        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f1)

        confusion = confusion_matrix(y_test, y_pred)
        print(confusion)

        accuracy_scores.append(accuracy)
        f1_scores.append(f1)
        confusion_matrices.append(confusion)

        report = classification_report(y_test, y_pred, zero_division=1)
        report_list.append(report)
        print(report)
        
        print(f" fold {fold+1} is Finished!")
        
        # save the output to a text file
        with open(f'{results_path}/dataset_{dataset_name}_ENCODER_fold_{fold+1}.txt', 'w') as f:
            f.write(f'Accuracy: {accuracy}\n')
            f.write(f'F1 Score: {f1}\n')
            f.write(f'Confusion Matrix:\n{confusion}\n\n')
            f.write(f'Classification report:\n{report}\n\n')
        
    with open(f'{results_path}/dataset_{dataset_name}_ENCODER.txt', 'w') as f:
        f.write("Mean accuracy: {:.3f} (std={:.3f})\n".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
        f.write("Mean F1 score: {:.3f} (std={:.3f})\n".format(np.mean(f1_scores), np.std(f1_scores)))
        f.write("Mean confusion matrix:\n{}\n".format(np.array2string(np.mean(confusion_matrices, axis=0))))
        f.write("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    print(" Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    



 The dataset shape is:(53785, 749, 3)

 The number of data samples (N) is:53785

 The number of TS length (T) is:749

 The number of TS dimention (M) is:3


/home/ma00048/miniconda3/envs/DL/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 749, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 749, 128)     2048        ['input_1[0][0]']                
                                                                                                  
 instance_normalization (Instan  (None, 749, 128)    256         ['conv1d[0][0]']                 
 ceNormalization)                                                                                 
                                                                                                  
 p_re_lu (PReLU)                (None, 749, 128)     128         ['instance_normalization[0][0

3586/3586 [==============================] - 34s 9ms/step - loss: 0.0130 - accuracy: 0.9968 - val_loss: 0.0108 - val_accuracy: 0.9976
Epoch 17/100
3586/3586 [==============================] - 33s 9ms/step - loss: 0.0122 - accuracy: 0.9966 - val_loss: 0.0103 - val_accuracy: 0.9975
Epoch 18/100
3586/3586 [==============================] - 34s 9ms/step - loss: 0.0109 - accuracy: 0.9971 - val_loss: 0.0112 - val_accuracy: 0.9978
Epoch 19/100
3586/3586 [==============================] - 33s 9ms/step - loss: 0.0110 - accuracy: 0.9970 - val_loss: 0.0143 - val_accuracy: 0.9964
Epoch 20/100
3586/3586 [==============================] - 34s 9ms/step - loss: 0.0109 - accuracy: 0.9970 - val_loss: 0.0114 - val_accuracy: 0.9973
Epoch 21/100
3586/3586 [==============================] - 33s 9ms/step - loss: 0.0092 - accuracy: 0.9976 - val_loss: 0.0090 - val_accuracy: 0.9983
Epoch 22/100
3586/3586 [==============================] - 33s 9ms/step - loss: 0.0106 - accuracy: 0.9972 - val_loss: 0.0127 - val_a

KeyboardInterrupt: 

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res

In [2]:
df_metrics = calculate_metrics(y_true, y_pred, 0.0)

NameError: name 'y_true' is not defined

In [ ]:
classifier.predict(X_test, y_true,X_train,y_train,y_test,return_df_metrics = True)

In [2]:
import h5py
f = h5py.File('./Results/PHM2022_Multivar_Datasetbest_model.hdf5', 'r')

In [4]:
list(f.keys())

['model_weights', 'optimizer_weights']